In [4]:
from dotenv import load_dotenv
load_dotenv()

import logging
import sys
date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

from falkordb_gemini_kg.classes.model_config import KnowledgeGraphModelConfig
from falkordb_gemini_kg import KnowledgeGraph, Ontology
from falkordb_gemini_kg.classes.source import Source
from falkordb_gemini_kg.models.openai import OpenAiGenerativeModel
import os
from random import shuffle
import json
from falkordb import FalkorDB

### Import source data

In [5]:
src_files = "data/fight"
sources = []

# For each file in the source directory, create a new Source object
for file in os.listdir(src_files):
    sources.append(Source(os.path.join(src_files, file)))

### Automatically create the ontology from the sources

In [6]:
boundaries = """
    Extract only the most information about the fighters, fights, and events in the UFC.
    Do not create entities for what can be expressed as attributes.
"""

model = OpenAiGenerativeModel(model_name="gpt-4o")

ontology = Ontology.from_sources(
    sources=sources[: round(len(sources) * 0.1)],
    boundaries=boundaries,
    model=model,
)


db = FalkorDB()
graph = db.select_graph("ufc_ontology")
ontology.save_to_graph(graph)

# Save ontology to json file
with open("ontologies/ufc_ontology.json", "w", encoding="utf-8") as file:
    file.write(json.dumps(ontology.to_json(), indent=2))

2024-08-24 13:25:48 HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-08-24 13:25:48 Model response: GenerationResponse(text=```json
{"entities":[{"label":"Person","attributes":[{"name":"name","type":"string","unique":true,"required":true}]},{"label":"Event","attributes":[{"name":"name","type":"string","unique":true,"required":true},{"name":"date","type":"string","unique":false,"required":true},{"name":"location","type":"string","unique":false,"required":true}]},{"label":"Fight","attributes":[{"name":"method","type":"string","unique":false,"required":true},{"name":"round","type":"number","unique":false,"required":true},{"name":"time","type":"string","unique":false,"required":true},{"name":"time_format","type":"string","unique":false,"required":true}]},{"label":"Referee","attributes":[{"name":"name","type":"string","unique":true,"required":true}]}],"relations":[{"label":"PARTICIPATED_IN","source":{"label":"Person"},"target":{"label":"Fight"}},{"label":

### Read ontology from json file

In [7]:
ontology_file = "ontologies/ufc_ontology_corrected.json"
with open(ontology_file, "r", encoding="utf-8") as file:
    ontology = Ontology.from_json(json.loads(file.read()))

db = FalkorDB()
graph = db.select_graph("ufc_ontology")
ontology.save_to_graph(graph)

kg = KnowledgeGraph(
    name="ufc",
    model_config=KnowledgeGraphModelConfig.with_model(model),
    ontology=ontology,
)

2024-08-24 13:31:17 Query: MERGE (n:Fighter {name: "string!*", nickname: "string!"}) SET n += {} RETURN n
2024-08-24 13:31:17 Query: MERGE (n:Fight {name: "string!*"}) SET n += {date: "string*", location: "string*", method: "string", time: "string", details: "string", rounds: "number", decision: "string", winner: "string", loser: "string"} RETURN n
2024-08-24 13:31:17 Query: MERGE (n:FightTotals {fight_name: "string!*", fighter: "string!*"}) SET n += {knockdowns: "number", significant_strikes: "number", significant_strikes_attempted: "number", significant_strikes_percentage: "number", total_strikes: "number", total_strikes_attempted: "number", takedowns: "number", takedowns_attempted: "number", takedown_percentage: "number", submissions_attempted: "number", passes: "number", reversals: "number"} RETURN n
2024-08-24 13:31:17 Query: MERGE (n:Event {name: "string!*"}) SET n += {location: "string*", date: "string*"} RETURN n
2024-08-24 13:31:17 Query: MERGE (n:WeightClass {name: "string!*"

### Process the sources raw data into the knowledge graph

In [8]:
kg.process_sources(sources)

2024-08-24 13:31:32 Processing 766 documents
2024-08-24 13:31:32 Processing task: extract_data_step_058c1a60-a0ee-4aef-8a41-0bf997732310
2024-08-24 13:31:32 Processing task: extract_data_step_08cdf134-1b19-499a-ab41-8ca48216134f
2024-08-24 13:31:32 Processing task: extract_data_step_25ee1869-f59e-4f42-a4ee-6d18fe160de6
2024-08-24 13:31:32 Processing task: extract_data_step_9aee0b95-0e7e-4f55-8498-ea7e7fba3270
2024-08-24 13:31:32 Processing task: extract_data_step_627dcbbe-8ea7-4384-9673-ea23846148ef
2024-08-24 13:31:32 Processing task: extract_data_step_058c1a60-a0ee-4aef-8a41-0bf997732310
2024-08-24 13:31:32 Processing task: extract_data_step_156a7666-d52e-4b43-8769-c6269f90f34b
2024-08-24 13:31:32 Processing task: extract_data_step_d5946c28-18f1-4c59-a213-793c3ac9ef2e
2024-08-24 13:31:32 Processing task: extract_data_step_3be99369-cf9e-43f3-8842-c7d202175558
2024-08-24 13:31:32 Processing task: extract_data_step_84c9302c-1331-4997-a029-7d6f4a532f53
2024-08-24 13:31:32 Processing task

### Ask a single question to the model

In [ ]:
kg.ask("What were the last 5 fights? When were they? How many rounds did they have?")

In [ ]:
kg.ask("How many takedowns did Alexandre Pantoja have in all fights?")

### Start a chat session with the model

In [ ]:
chat = kg.chat_session()

print(chat.send_message("Who is Salsa Boy?"))